## Imports

In [19]:
from google.cloud import bigquery
import pandas as pd

In [6]:
import plotly.plotly as py
import plotly.tools as tools
import plotly.figure_factory as ff
import plotly.graph_objs as go

## Config

In [ ]:
project= 'YOUR_PROJECT'
dataset = 'YOUR_DATASET'
table = 'YOUR_TABLE'

In [3]:
client = bigquery.Client(project=project)

In [ ]:
tools.set_credentials_file(username='USERNAME', api_key='APIKEY')

## Get data

In [130]:
query="""
SELECT
  CAST(ROUND(AVG(sentiment_norm),4) AS STRING) as avg_sentiment,
  count(*) as number_posts,
  posted_at
FROM(
  SELECT 
  DISTINCT
  text, 
  id,
  sentiment,
  (sentiment-mean_sentiment) / std_sentiment as sentiment_norm,
  TIMESTAMP_TRUNC(posted_at, MINUTE) as posted_at
FROM `{project}.{dataset}.{table}` 
LEFT JOIN (SELECT AVG(sentiment) as mean_sentiment, stddev(sentiment) as std_sentiment FROM `{project}.{dataset}.{table}`) ON True
WHERE TIMESTAMP_TRUNC(posted_at, HOUR) IN (
  CAST("2019-05-20 02:00:00 UTC" AS TIMESTAMP),
  CAST("2019-05-20 03:00:00 UTC" AS TIMESTAMP),
  CAST("2019-05-20 04:00:00 UTC" AS TIMESTAMP),
  CAST("2019-05-20 05:00:00 UTC" AS TIMESTAMP))) 
GROUP BY posted_at
ORDER BY posted_at ASC
"""

In [131]:
df_data = client.query(query).result().to_dataframe()

In [132]:
df_data.head()

,avg_sentiment,number_posts,posted_at
0,-0.2402,1792,2019-05-20 02:00:00+00:00
1,-0.2352,1758,2019-05-20 02:01:00+00:00
2,-0.2542,1661,2019-05-20 02:02:00+00:00
3,-0.2678,1764,2019-05-20 02:03:00+00:00
4,-0.1698,1695,2019-05-20 02:04:00+00:00


## Calculate the moving mean sentiment

In [133]:
df_data["moving_mean"] = df_data.avg_sentiment.rolling(window =5).mean()

In [134]:
df_data.moving_mean = df_data.moving_mean.combine_first(df_data.avg_sentiment)

## Plot

In [144]:
traces = []

traces.append(go.Scatter(
            x=df_data.posted_at,
            y=df_data.moving_mean,
            name="moving average"
        ))

traces.append(go.Scatter(
            x=df_data.posted_at,
            y=df_data.avg_sentiment,
            name="raw sentiment per minute",
            opacity=0.25
        ))

In [151]:
layout = go.Layout(
        title='Mean standardized sentiment over time',
        xaxis=dict(
            title='time',
            titlefont=dict(
                size=18,
                color='#7f7f7f'
            )
        ),
        yaxis=dict(
            title='mean standardized sentiment',
            titlefont=dict(
                size=18,
                color='#7f7f7f'
            )
        ),
        annotations=[
        dict(
            x='2019-05-20 03:00:00',
            y=float(df_data[df_data.posted_at=='2019-05-20 03:00:00'].moving_mean),
            xref='x',
            yref='y',
            text='start of the episode',
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=-40
        ),
        dict(
            x='2019-05-20 03:18:00',
            y=float(df_data[df_data.posted_at=='2019-05-20 03:18:00'].moving_mean),
            xref='x',
            yref='y',
            text='tyrion defies dany',
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=50
        ),
        dict(
            x='2019-05-20 03:24:00',
            y=float(df_data[df_data.posted_at=='2019-05-20 03:24:00'].moving_mean),
            xref='x',
            yref='y',
            text='tyrion speaks to jon',
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=-40
        ),
        dict(
            x='2019-05-20 03:50:00',
            y=float(df_data[df_data.posted_at=='2019-05-20 03:50:00'].moving_mean),
            xref='x',
            yref='y',
            text='bran is named king',
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=-40
        ),
        dict(
            x='2019-05-20 03:36:00',
            y=float(df_data[df_data.posted_at=='2019-05-20 03:36:00'].moving_mean),
            xref='x',
            yref='y',
            text='jon stabs dany',
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=90
        ),
        dict(
            x='2019-05-20 04:05:00',
            y=float(df_data[df_data.posted_at=='2019-05-20 04:05:00'].moving_mean),
            xref='x',
            yref='y',
            text='ser brienne writes about ser jaime',
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=120
        ),
        dict(
            x='2019-05-20 04:18:00',
            y=float(df_data[df_data.posted_at=='2019-05-20 04:18:00'].moving_mean),
            xref='x',
            yref='y',
            text='end of the episode',
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=-60
        )
        ]
    )


In [152]:
fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='td_medium_twitter_sentiment_ma')